In [1]:
# Importing the libraries
from bs4 import BeautifulSoup as BS
from lxml import html
import pandas as pd
from time import sleep
from selenium import webdriver

In [2]:
# To run the program smoothly
def open_browser(alt_user_name = 'Thank you for your website'):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('window-size=1920x1080')
    chrome_options.add_argument(f'user-agent={alt_user_name}')
    driver = webdriver.Chrome('/Users/ab/Downloads/chromedriver')
    return driver

In [3]:
def number(enter):
    no=enter.index(":")
    return(enter[no+2:])

def clean(record):
    g=record.split()
    return (g[0])

In [22]:

base_url="https://www.trulia.com"
city=[
    "https://www.trulia.com/for_sale/Mountain_View,CA/1p_beds/SINGLE-FAMILY_HOME_type/",
    "https://www.trulia.com/for_sale/Santa_Clara,CA/1p_beds/SINGLE-FAMILY_HOME_type/",
    "https://www.trulia.com/for_sale/San_Francisco,CA/1p_beds/SINGLE-FAMILY_HOME_type/",
    "https://www.trulia.com/for_sale/Palo_Alto,CA/1p_beds/SINGLE-FAMILY_HOME_type/",
    "https://www.trulia.com/for_sale/Fremont,CA/1p_beds/SINGLE-FAMILY_HOME_type/",
    "https://www.trulia.com/for_sale/San_Jose,CA/1p_beds/SINGLE-FAMILY_HOME_type/",
]
#city = ["https://www.trulia.com/for_sale/San_Jose,CA/1p_beds/SINGLE-FAMILY_HOME_type/"]


In [23]:
driver = open_browser()
output =[]


In [25]:

for i in city: # to extract the url for each property
    print(i)
    driver.get(i)
    source_code = driver.page_source
    soup_data = BS(source_code, 'lxml')
    all_urls = [a['href'] for a in soup_data('a')  if a.has_attr('href')]

    extracted_url=list(set([s for s in all_urls if "/p/ca/" in s]))
    counter=0
    for personal_url in extracted_url: # to create the url extract each property and their respective information 
        url=f"{base_url}{personal_url}"
        data={}
    
        try:
            print(url)
            driver.get(url)
            sourcecode1 = driver.page_source
            soup1 = BS(sourcecode1, 'lxml')

            try:
                data["Street_Address"]=(soup1.find("span",{"data-testid":"home-details-summary-headline"}).text)
            except:
                pass
            try:
                loc=(soup1.find("span",{"data-testid":"home-details-summary-city-state"}).text)
                data["City"]=loc[:loc.index(",")]
            except:
                pass
            try:
                loc=(soup1.find("span",{"data-testid":"home-details-summary-city-state"}).text)
                data["Zipcode"]=loc[loc.index("CA")+3:]
            except:
                pass


            fact=soup1.find("div",{"data-testid":"home-details-summary-container"})
            try:
                data["Bed"]=int(clean(fact.find("div", {"data-testid":"home-summary-size-bedrooms"}).text))#bedroom
            except:
                pass

            try:
                data["Bath"]=float(clean(fact.find("div", {"data-testid":"home-summary-size-bathrooms"}).text))
            except:
                pass
            try:
                data["Floor_Area(sq.ft)"]=int(clean(fact.find("div", {"data-testid":"home-summary-size-floorspace"}).text.replace(",","")))#bedroom
            except:
                pass

            try:
                yo= soup1.find("h3",{"data-testid":"on-market-price-details"}).text.replace(",","")
                data["Price($)"]=float(yo[1:])
            except:
                pass

            try:
                num=soup1.find("div",{"data-testid":"summary-mortgage-estimate"}).text## Estimated_Mortgage/month ($)
                data["Estimated_mortgage/mo($)"]=float(num[(num.index("$"))+1:num.index("/mo")].replace(",",""))
            except:
                pass


            explore_area=soup1.find("div", {"data-testid":"explore-area-container"})
            sch=explore_area.find_all("div",{"data-testid":"card-description"})

            for i in range(0,len(sch)):
                if "elementary school" in sch[i].text.lower():
                    data["Elementary_School"]=int(clean(sch[i].text))
                elif "middle school" in sch[i].text.lower():
                    data["Middle_school"]=int(clean(sch[i].text))
                elif "high school" in sch[i].text.lower():
                    data["High_School"]=int(clean(sch[i].text))
                elif "crime" in sch[i].text.lower():
                    data["Crime"]= clean(sch[i].text)
                elif "residents commute" in sch[i].text.lower():
                    commute= clean(sch[i].text)
                    data["Commute_By_Car(%)"]=float(commute[:len(commute)-1])
                elif "restaurants" in sch[i].text.lower():
                    data["Restaurants"]=int(clean(sch[i].text))
                elif "groceries" in sch[i].text.lower():
                    data["Groceries"]=int(clean(sch[i].text))
                elif "nightlife" in sch[i].text.lower():
                    data["Nightlife"]=int(clean(sch[i].text))



            home_all=soup1.find_all("table",{ "class":"Table-latbb5-3 lomxvq"})
            for op in range(0,len(home_all)):
                heading=home_all[op].find_all("div", {"class":"Text__TextBase-sc-1cait9d-0-div Text__TextContainerBase-sc-1cait9d-1 juzGEy"})
                for hi in heading:
                    if hi.text.lower()=="property information":
                        subheading=home_all[op].find("div", {"data-testid":"structured-amenities-table-subcategory"})
                        if(subheading.text.lower() == "year built"):
                            data["Year_Built"]=number(home_all[op].find("span", {"class":"Feature__FeatureListItem-sc-8qiunc-0 kLjNvv"}).text)

                    elif hi.text.lower()=="lot information": 
                         data["lot_Size(sq.ft)"]=float((((home_all[op].find("span", {"class":"Feature__FeatureListItem-sc-8qiunc-0 kLjNvv"}).text).split())[2]).replace(",",""))


            try:
                taxInfoTable = soup1.find("div",{"data-testid":"property-tax-container"})
                taxRow=taxInfoTable.find_all("tr")
                for row in taxRow:
                    header = row.find("th").text
                    if(header.lower() == "tax"):
                        data["Property_Tax_2020"] = float(row.find("td").text.replace("$","").replace(",",""))
                    elif(header.lower() == "assessment"):
                        data["Property_Assessment($)"] = float(row.find("td").text.replace("$","").replace(",",""))

            except:
                pass
            output.append(data)
            print(data)
        except:
            pass
        sleep(5)    
    sleep(15)

https://www.trulia.com/for_sale/Mountain_View,CA/1p_beds/SINGLE-FAMILY_HOME_type/
https://www.trulia.com/p/ca/mountain-view/1183-solana-dr-mountain-view-ca-94040--2082707118
{'Street_Address': '1183 Solana Dr', 'City': 'Mountain View', 'Zipcode': '94040', 'Bed': 4, 'Bath': 2.0, 'Floor_Area(sq.ft)': 1501, 'Price($)': 2425000.0, 'Estimated_mortgage/mo($)': 9551.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 89.0, 'Restaurants': 139, 'Groceries': 17, 'Nightlife': 20, 'Year_Built': '1951', 'lot_Size(sq.ft)': 6250.0, 'Property_Tax_2020': 6451.0, 'Property_Assessment($)': 519970.0}
https://www.trulia.com/p/ca/palo-alto/1031-embarcadero-rd-palo-alto-ca-94303--2083118283
{'Street_Address': '1031 Embarcadero Rd', 'City': 'Palo Alto', 'Zipcode': '94303', 'Bed': 7, 'Bath': 6.0, 'Floor_Area(sq.ft)': 2842, 'Price($)': 3499888.0, 'Estimated_mortgage/mo($)': 13751.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowes

https://www.trulia.com/p/ca/los-altos/12200-s-el-monte-rd-los-altos-ca-94022--2117099161
{'Street_Address': '12200 S El Monte Rd', 'City': 'Los Altos', 'Zipcode': '94022', 'Bed': 4, 'Bath': 4.0, 'Floor_Area(sq.ft)': 3220, 'Price($)': 3998000.0, 'Estimated_mortgage/mo($)': 15697.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 96.0, 'Restaurants': 2, 'Year_Built': '1977', 'lot_Size(sq.ft)': 1.62}
https://www.trulia.com/p/ca/los-altos/1289-eureka-ave-los-altos-ca-94024--2082672307
{'Street_Address': '1289 Eureka Ave', 'City': 'Los Altos', 'Zipcode': '94024', 'Bed': 5, 'Bath': 5.0, 'Floor_Area(sq.ft)': 3007, 'Price($)': 3750000.0, 'Estimated_mortgage/mo($)': 14728.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 96.0, 'Restaurants': 13, 'Groceries': 4, 'Nightlife': 1, 'Year_Built': '2019', 'lot_Size(sq.ft)': 10323.0, 'Property_Tax_2020': 45280.0, 'Property_Assessment($)': 365241

{'Street_Address': '197 Thompson Sq', 'City': 'Mountain View', 'Zipcode': '94043', 'Bed': 3, 'Bath': 2.0, 'Floor_Area(sq.ft)': 1389, 'Price($)': 1998000.0, 'Estimated_mortgage/mo($)': 7882.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 92.0, 'Restaurants': 107, 'Groceries': 15, 'Nightlife': 15, 'Year_Built': '1956', 'lot_Size(sq.ft)': 6499.0, 'Property_Tax_2020': 13492.0, 'Property_Assessment($)': 1081579.0}
https://www.trulia.com/p/ca/mountain-view/1104-phyllis-ave-mountain-view-ca-94040--2082706652
{'Street_Address': '1104 Phyllis Ave', 'City': 'Mountain View', 'Zipcode': '94040', 'Bed': 3, 'Bath': 2.0, 'Floor_Area(sq.ft)': 1305, 'Price($)': 1995000.0, 'Estimated_mortgage/mo($)': 7871.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 89.0, 'Restaurants': 141, 'Groceries': 15, 'Nightlife': 17, 'Year_Built': '1956', 'lot_Size(sq.ft)': 6067.0, 'Property_Tax_2020': 1276.0, 'Pr

{'Street_Address': '1304 Todd St', 'City': 'Mountain View', 'Zipcode': '94040', 'Bed': 3, 'Bath': 2.0, 'Floor_Area(sq.ft)': 1273, 'Price($)': 2488000.0, 'Estimated_mortgage/mo($)': 9797.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 89.0, 'Restaurants': 144, 'Groceries': 14, 'Nightlife': 18, 'Year_Built': '1954', 'lot_Size(sq.ft)': 6006.0, 'Property_Tax_2020': 21261.0, 'Property_Assessment($)': 1681338.0}
https://www.trulia.com/p/ca/palo-alto/4206-pomona-ave-palo-alto-ca-94306--2083136212
{'Street_Address': '4206 Pomona Ave', 'City': 'Palo Alto', 'Zipcode': '94306', 'Bed': 5, 'Bath': 6.0, 'Floor_Area(sq.ft)': 3564, 'Price($)': 5380000.0, 'Estimated_mortgage/mo($)': 21098.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 83.0, 'Restaurants': 84, 'Groceries': 14, 'Nightlife': 12, 'Year_Built': '2021', 'lot_Size(sq.ft)': 9387.0, 'Property_Tax_2020': 34685.0, 'Property_Assessmen

https://www.trulia.com/p/ca/santa-clara/3816-shasta-dr-santa-clara-ca-95051--2084325155
{'Street_Address': '3816 Shasta Dr', 'City': 'Santa Clara', 'Zipcode': '95051', 'Bed': 4, 'Bath': 2.0, 'Floor_Area(sq.ft)': 1712, 'Price($)': 1950888.0, 'Estimated_mortgage/mo($)': 7698.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 95.0, 'Restaurants': 109, 'Groceries': 11, 'Nightlife': 11, 'Year_Built': '1959', 'lot_Size(sq.ft)': 7291.0, 'Property_Tax_2020': 2486.0, 'Property_Assessment($)': 197888.0}
https://www.trulia.com/p/ca/santa-clara/3025-dibble-ct-santa-clara-ca-95051--2084320869
{'Street_Address': '3025 Dibble Ct', 'City': 'Santa Clara', 'Zipcode': '95051', 'Bed': 4, 'Bath': 2.0, 'Floor_Area(sq.ft)': 1092, 'Price($)': 1378000.0, 'Estimated_mortgage/mo($)': 5459.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 95.0, 'Restaurants': 101, 'Groceries': 12, 'Nightlife': 8, 'Year_Bui

...

{'Street_Address': '52 Park Belmont Pl', 'City': 'San Jose', 'Zipcode': '95136', 'Bed': 4, 'Bath': 3.0, 'Floor_Area(sq.ft)': 2124, 'Price($)': 1120000.0, 'Estimated_mortgage/mo($)': 4447.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 96.0, 'Restaurants': 53, 'Groceries': 12, 'Nightlife': 6, 'Year_Built': '1978', 'lot_Size(sq.ft)': 6851.0, 'Property_Tax_2020': 11582.0, 'Property_Assessment($)': 767681.0}
https://www.trulia.com/p/ca/san-jose/219-avenida-espana-san-jose-ca-95139--2084611777
{'Street_Address': '219 Avenida Espana', 'City': 'San Jose', 'Zipcode': '95139', 'Bed': 4, 'Bath': 3.0, 'Floor_Area(sq.ft)': 1774, 'Price($)': 1249800.0, 'Estimated_mortgage/mo($)': 4953.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 95.0, 'Restaurants': 21, 'Groceries': 2, 'Nightlife': 3, 'Year_Built': '1975', 'lot_Size(sq.ft)': 6133.0, 'Property_Tax_2020': 9562.0, 'Property_Assessment($

https://www.trulia.com/p/ca/san-jose/1876-bird-ave-san-jose-ca-95125--2084516153
{'Street_Address': '1876 Bird Ave', 'City': 'San Jose', 'Zipcode': '95125', 'Bed': 3, 'Bath': 1.0, 'Floor_Area(sq.ft)': 1161, 'Price($)': 979000.0, 'Estimated_mortgage/mo($)': 3896.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 94.0, 'Restaurants': 108, 'Groceries': 15, 'Nightlife': 15, 'Year_Built': '1950', 'lot_Size(sq.ft)': 4883.0, 'Property_Tax_2020': 2135.0, 'Property_Assessment($)': 64591.0}
https://www.trulia.com/p/ca/san-jose/1851-patio-dr-san-jose-ca-95125--2084516037
{'Street_Address': '1851 Patio Dr', 'City': 'San Jose', 'Zipcode': '95125', 'Bed': 5, 'Bath': 6.0, 'Floor_Area(sq.ft)': 4068, 'Price($)': 3298000.0, 'Estimated_mortgage/mo($)': 12948.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 94.0, 'Restaurants': 102, 'Groceries': 13, 'Nightlife': 12, 'Year_Built': '2013', 'lot_Size

https://www.trulia.com/p/ca/san-jose/5646-wells-ct-san-jose-ca-95123--2084483575
{'Street_Address': '5646 Wells Ct', 'City': 'San Jose', 'Zipcode': '95123', 'Bed': 4, 'Bath': 2.0, 'Floor_Area(sq.ft)': 1883, 'Price($)': 985000.0, 'Estimated_mortgage/mo($)': 3920.0, 'Elementary_School': 1, 'Middle_school': 2, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 95.0, 'Restaurants': 63, 'Groceries': 9, 'Nightlife': 1, 'Year_Built': '1968', 'lot_Size(sq.ft)': 7130.0, 'Property_Tax_2020': 6222.0, 'Property_Assessment($)': 349350.0}
https://www.trulia.com/p/ca/san-jose/5458-morningside-dr-san-jose-ca-95138--2084608669
{'Street_Address': '5458 Morningside Dr', 'City': 'San Jose', 'Zipcode': '95138', 'Bed': 5, 'Bath': 8.0, 'Floor_Area(sq.ft)': 12008, 'Price($)': 14500000.0, 'Estimated_mortgage/mo($)': 56824.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 98.0, 'Restaurants': 3, 'Nightlife': 1, 'Year_Built': '2006', 'lot_Size(sq.ft)'

https://www.trulia.com/p/ca/san-jose/1385-palm-st-san-jose-ca-95110--1151029999
{'Street_Address': '1385 Palm St', 'City': 'San Jose', 'Zipcode': '95110', 'Bed': 4, 'Bath': 3.0, 'Floor_Area(sq.ft)': 1645, 'Price($)': 1178888.0, 'Estimated_mortgage/mo($)': 4676.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Moderate', 'Commute_By_Car(%)': 83.0, 'Restaurants': 139, 'Groceries': 29, 'Nightlife': 24, 'Year_Built': '2015', 'lot_Size(sq.ft)': 2901.0, 'Property_Tax_2020': 11315.0, 'Property_Assessment($)': 865943.0}
https://www.trulia.com/p/ca/san-jose/4102-crescendo-ave-san-jose-ca-95136--2084600272
{'Street_Address': '4102 Crescendo Ave', 'City': 'San Jose', 'Zipcode': '95136', 'Bed': 3, 'Bath': 2.0, 'Floor_Area(sq.ft)': 1305, 'Price($)': 999888.0, 'Estimated_mortgage/mo($)': 4011.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 96.0, 'Restaurants': 55, 'Groceries': 11, 'Nightlife': 4, 'Year_Built': '1972', 

{'Street_Address': '829 Nevada Ave', 'City': 'San Jose', 'Zipcode': '95125', 'Bed': 3, 'Bath': 2.0, 'Floor_Area(sq.ft)': 1392, 'Price($)': 1899000.0, 'Estimated_mortgage/mo($)': 7488.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 94.0, 'Restaurants': 102, 'Groceries': 19, 'Nightlife': 15, 'Year_Built': '1936', 'lot_Size(sq.ft)': 5762.0, 'Property_Tax_2020': 19243.0, 'Property_Assessment($)': 1540200.0}
https://www.trulia.com/p/ca/san-jose/1092-jayden-ln-san-jose-ca-95120--2411895093
{'Street_Address': '1092 Jayden Ln', 'City': 'San Jose', 'Zipcode': '95120', 'Bed': 4, 'Bath': 3.0, 'Floor_Area(sq.ft)': 2207, 'Price($)': 1768800.0, 'Estimated_mortgage/mo($)': 7159.0, 'Elementary_School': 1, 'Middle_school': 1, 'High_School': 1, 'Crime': 'Lowest', 'Commute_By_Car(%)': 96.0, 'Restaurants': 16, 'Groceries': 3, 'Nightlife': 3, 'Year_Built': '2018', 'lot_Size(sq.ft)': 4007.0, 'Property_Tax_2020': 13340.0, 'Property_Assessment($)': 1124

In [26]:
import pandas as pd

df= pd.DataFrame(output)
df

,Street_Address,City,Zipcode,Bed,Bath,Floor_Area(sq.ft),Price($),Estimated_mortgage/mo($),Elementary_School,Middle_school,High_School,Crime,Commute_By_Car(%),Restaurants,Groceries,Nightlife,Year_Built,lot_Size(sq.ft),Property_Tax_2020,Property_Assessment($)
0,1183 Solana Dr,Mountain View,94040,4,2.0,1501.0,2425000.0,9551.0,1.0,1.0,1.0,Lowest,89.0,139.0,17.0,20.0,1951,6250.00,6451.0,519970.0
1,1031 Embarcadero Rd,Palo Alto,94303,7,6.0,2842.0,3499888.0,13751.0,1.0,1.0,1.0,Lowest,90.0,29.0,6.0,3.0,2021,6050.00,25499.0,2080800.0
2,764 Alice Ave,Mountain View,94041,4,3.0,1841.0,2208888.0,8706.0,1.0,1.0,1.0,Lowest,85.0,158.0,18.0,20.0,1953,9877.00,1175.0,72187.0
3,798 W Iowa Ave,Sunnyvale,94086,5,4.0,2676.0,2398000.0,9446.0,1.0,1.0,1.0,Lowest,92.0,183.0,18.0,27.0,1955,10262.00,NaN,NaN
4,1885 Peacock Ave,Mountain View,94043,4,4.0,2076.0,2699000.0,10622.0,1.0,1.0,1.0,Lowest,92.0,166.0,14.0,25.0,2016,6046.00,30974.0,2523000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,1092 Jayden Ln,San Jose,95120,4,3.0,2207.0,1768800.0,7159.0,1.0,1.0,1.0,Lowest,96.0,16.0,3.0,3.0,2018,4007.00,13340.0,1124345.0
580,3690 Willowpark Dr,San Jose,95118,4,2.0,1426.0,1299000.0,5146.0,1.0,1.0,1.0,Lowest,96.0,52.0,12.0,6.0,1958,6377.00,2147.0,71137.0
581,1015 Noble Ave,San Jose,95132,4,3.0,2266.0,1588000.0,6274.0,1.0,1.0,1.0,Lowest,98.0,8.0,5.0,2.0,1967,0.31,17008.0,1246919.0
582,1115 Carolyn Ave,San Jose,95125,5,4.0,3771.0,2999000.0,11781.0,1.0,1.0,1.0,Lowest,94.0,76.0,21.0,15.0,2006,6351.00,28323.0,2280113.0


In [27]:
#removing duplicate records
df_new=df.drop_duplicates()
df_new

,Street_Address,City,Zipcode,Bed,Bath,Floor_Area(sq.ft),Price($),Estimated_mortgage/mo($),Elementary_School,Middle_school,High_School,Crime,Commute_By_Car(%),Restaurants,Groceries,Nightlife,Year_Built,lot_Size(sq.ft),Property_Tax_2020,Property_Assessment($)
0,1183 Solana Dr,Mountain View,94040,4,2.0,1501.0,2425000.0,9551.0,1.0,1.0,1.0,Lowest,89.0,139.0,17.0,20.0,1951,6250.00,6451.0,519970.0
1,1031 Embarcadero Rd,Palo Alto,94303,7,6.0,2842.0,3499888.0,13751.0,1.0,1.0,1.0,Lowest,90.0,29.0,6.0,3.0,2021,6050.00,25499.0,2080800.0
2,764 Alice Ave,Mountain View,94041,4,3.0,1841.0,2208888.0,8706.0,1.0,1.0,1.0,Lowest,85.0,158.0,18.0,20.0,1953,9877.00,1175.0,72187.0
3,798 W Iowa Ave,Sunnyvale,94086,5,4.0,2676.0,2398000.0,9446.0,1.0,1.0,1.0,Lowest,92.0,183.0,18.0,27.0,1955,10262.00,NaN,NaN
4,1885 Peacock Ave,Mountain View,94043,4,4.0,2076.0,2699000.0,10622.0,1.0,1.0,1.0,Lowest,92.0,166.0,14.0,25.0,2016,6046.00,30974.0,2523000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,1092 Jayden Ln,San Jose,95120,4,3.0,2207.0,1768800.0,7159.0,1.0,1.0,1.0,Lowest,96.0,16.0,3.0,3.0,2018,4007.00,13340.0,1124345.0
580,3690 Willowpark Dr,San Jose,95118,4,2.0,1426.0,1299000.0,5146.0,1.0,1.0,1.0,Lowest,96.0,52.0,12.0,6.0,1958,6377.00,2147.0,71137.0
581,1015 Noble Ave,San Jose,95132,4,3.0,2266.0,1588000.0,6274.0,1.0,1.0,1.0,Lowest,98.0,8.0,5.0,2.0,1967,0.31,17008.0,1246919.0
582,1115 Carolyn Ave,San Jose,95125,5,4.0,3771.0,2999000.0,11781.0,1.0,1.0,1.0,Lowest,94.0,76.0,21.0,15.0,2006,6351.00,28323.0,2280113.0


In [28]:
df_new.to_csv("Project_Data.csv")